# Matrix vector multiplication in SHEEP
Since we can use "slots", i.e. vectors of input values, and many libraries support SIMD operations (i.e. operating on many slots is just as quick as operating on single values), we can easily do component-wise multiplication.  However, as with vector dot products, we generally need to also sum over slots, which can be done using a sequence of "ROTATE" and "ADD" operations.

In [57]:
import os
if "SHEEP_HOME" in os.environ.keys():
  SHEEP_HOME = os.environ["SHEEP_HOME"]
else:
  SHEEP_HOME = os.path.join(os.environ["HOME"],"SHEEP","frontend")
import sys
sys.path.append(SHEEP_HOME)

from pysheep import sheep_client

## Multiplying a 4x4 matrix with a 4-component vector


Lets do the following calculation:
\begin{equation*}
\begin{vmatrix}
1, 2 , 3 , 4\\
5 , 6 , 7 , 8\\
9, 10, 11, 12 \\
13, 14 , 15, 16
\end{vmatrix}
\begin{vmatrix}
1 \\
2 \\
3 \\
4
\end{vmatrix}
\end{equation*}

which should give us the answer {30, 55, 110, 150}


### a) The straightforward (but non-optimal) way

Essentially the each element of the output vector will be the dot product of the corresponding row of the matrix with the vector.  We can therefore do this in the same way as was demonstrated in the vector_dot_product notebook, i.e. component-wise multiplication followed by a sequence of rotations and additions.

The circuit will look like:

In [77]:
circuit = """
INPUTS mrow_0 mrow_1 mrow_2 mrow_3 vec mask
CONST_INPUTS rotate_by_minus1 rotate_by_plus1 rotate_by_plus2 rotate_by_plus3
OUTPUTS output_vec 
# dot product of the first row with the vector
mrow_0 vec MULTIPLY prod_00
prod_00 rotate_by_minus1 ROTATE prod_01
prod_00 prod_01 ADD sum_00
prod_01 rotate_by_minus1 ROTATE prod_02
sum_00 prod_02 ADD sum_01
prod_02 rotate_by_minus1 ROTATE prod_03
sum_01 prod_03 ADD sum_0
# dot product of the second row with the vector
mrow_1 vec MULTIPLY prod_10
prod_10 rotate_by_minus1 ROTATE prod_11
prod_10 prod_11 ADD sum_10
prod_11 rotate_by_minus1 ROTATE prod_12
sum_10 prod_12 ADD sum_11
prod_12 rotate_by_minus1 ROTATE prod_13
sum_11 prod_13 ADD sum_1
# dot product of the third row with the vector
mrow_2 vec MULTIPLY prod_20
prod_20 rotate_by_minus1 ROTATE prod_21
prod_20 prod_21 ADD sum_20
prod_21 rotate_by_minus1 ROTATE prod_22
sum_20 prod_22 ADD sum_21
prod_22 rotate_by_minus1 ROTATE prod_23
sum_21 prod_23 ADD sum_2
# dot product of the third row with the vector
mrow_3 vec MULTIPLY prod_30
prod_30 rotate_by_minus1 ROTATE prod_31
prod_30 prod_31 ADD sum_30
prod_31 rotate_by_minus1 ROTATE prod_32
sum_30 prod_32 ADD sum_31
prod_32 rotate_by_minus1 ROTATE prod_33
sum_31 prod_33 ADD sum_3
# now we have four vectors, sum_0, sum_1, sum_2 and sum_3, where the first element is 
# the dot product of that row.  We need to isolate just this element, using mask, which is [1,0,0,0]
sum_0 mask MULTIPLY msum_0
sum_1 mask MULTIPLY msum_10
msum_10 rotate_by_plus1 ROTATE msum_1
sum_2 mask MULTIPLY msum_20
msum_20 rotate_by_plus2 ROTATE msum_2
sum_3 mask MULTIPLY msum_30
msum_30 rotate_by_plus3 ROTATE msum_3
# now we should have four vectors with one non-zero element each in the right place - need to sum them
msum_0 msum_1 ADD out_01
out_01 msum_2 ADD out_02
out_02 msum_3 ADD output_vec
"""

In [78]:
sheep_client.new_job()
#sheep_client.set_context("HElib_Fp")
sheep_client.set_context("HElib_Fp")
sheep_client.set_input_type("int16_t")
#sheep_client.set_parameters({"NumSlots": 4})
#sheep_client.set_parameters({"BaseParamSet": 2})
sheep_client.set_circuit_text(circuit)
sheep_client.get_inputs()

{'content': ['mrow_0', 'mrow_1', 'mrow_2', 'mrow_3', 'vec', 'mask'],
 'status_code': 200}

In [79]:
sheep_client.get_const_inputs()

{'content': ['rotate_by_minus1',
  'rotate_by_plus1',
  'rotate_by_plus2',
  'rotate_by_plus3'],
 'status_code': 200}

In [80]:
sheep_client.set_inputs({"mrow_0": [1,2,3,4], "mrow_1": [5,6,7,8], "mrow_2": [9,10,11,12], "mrow_3": [13,14,15,16],"vec": [1,2,3,4], "mask": [1,0,0,0]})


{'content': '', 'status_code': 200}

In [81]:
sheep_client.set_const_inputs({"rotate_by_minus1": -1, "rotate_by_plus1": 1, "rotate_by_plus2": 2, "rotate_by_plus3":3})

{'content': '', 'status_code': 200}

In [82]:
sheep_client.run_job()

{'content': '', 'status_code': 200}

In [83]:
sheep_client.get_results()

{'content': {'cleartext check': {'is_correct': False},
  'outputs': {'output_vec': ['15177,-4917,8134,13412']},
  'timings': {'decryption': '188.071000',
   'encryption': '1577.778000',
   'evaluation': '18309.498000',
   'msum_0': '73.508000',
   'msum_1': '58.284000',
   'msum_10': '58.461000',
   'msum_2': '58.789000',
   'msum_20': '59.888000',
   'msum_3': '58.525000',
   'msum_30': '52.876000',
   'out_01': '58.070000',
   'out_02': '59.961000',
   'output_vec': '73.531000',
   'prod_00': '419.122000',
   'prod_01': '757.893000',
   'prod_02': '494.935000',
   'prod_03': '794.257000',
   'prod_10': '51.905000',
   'prod_11': '79.247000',
   'prod_12': '603.903000',
   'prod_13': '752.294000',
   'prod_20': '259.813000',
   'prod_21': '915.882000',
   'prod_22': '736.736000',
   'prod_23': '896.536000',
   'prod_30': '708.984000',
   'prod_31': '248.487000',
   'prod_32': '1014.472000',
   'prod_33': '74.490000',
   'sum_0': '449.708000',
   'sum_00': '503.757000',
   'sum_01': '4

### b) A better way -  doing the same calculation with fewer ROTATEs.
To minimize the number of intra-slot operations, we can instead express the first three inputs as diagonal strips of the matrix.

In this case, the circuit will look like:

In [71]:
new_circuit = """
INPUTS mstrip_0 mstrip_1 mstrip_2 mstrip_3 vec 
CONST_INPUTS rotate_minus1
OUTPUTS output_vec 
mstrip_0 vec MULTIPLY prod_0
vec rotate_minus1 ROTATE vec_r1
mstrip_1 vec_r1 MULTIPLY prod_1
vec_r1 rotate_minus1 ROTATE vec_r2
mstrip_2 vec_r2 MULTIPLY prod_2
vec_r2 rotate_minus1 ROTATE vec_r3
mstrip_3 vec_r3 MULTIPLY prod_3
prod_0 prod_1 ADD sum_0
sum_0 prod_2 ADD sum_1
sum_1 prod_3 ADD output_vec
"""

In [72]:
sheep_client.new_job()
sheep_client.set_context("HElib_Fp")
sheep_client.set_input_type("int16_t")
sheep_client.set_circuit_text(new_circuit)


{'content': '', 'status_code': 200}

We can set some of the inputs similarly to above - the "mask" ones are [1,0,0], [0,1,0], [0,0,1] respectively

In [73]:
const_input_vals = {"rotate_minus1": -1}
input_vals = {"vec": [1,2,3,4]}

So now we have to set the remaining inputs to be diagonal strips of the matrix:

In [74]:
input_vals["mstrip_0"] = [1,6,11,16]
input_vals["mstrip_1"] = [2,7,12,13]
input_vals["mstrip_2"] = [3,8,9,14]
input_vals["mstrip_3"] = [4,5,10,15]
sheep_client.set_inputs(input_vals)
sheep_client.set_const_inputs(const_input_vals)

{'content': '', 'status_code': 200}

In [75]:
sheep_client.run_job()

{'content': '', 'status_code': 200}

In [76]:
sheep_client.get_results()

{'content': {'cleartext check': {'is_correct': True},
  'outputs': {'output_vec': ['30,70,110,150']},
  'timings': {'decryption': '143.833000',
   'encryption': '1292.325000',
   'evaluation': '4458.559000',
   'output_vec': '783.767000',
   'prod_0': '51.385000',
   'prod_1': '335.034000',
   'prod_2': '351.104000',
   'prod_3': '55.641000',
   'sum_0': '745.061000',
   'sum_1': '799.979000',
   'vec_r1': '514.601000',
   'vec_r2': '325.915000',
   'vec_r3': '46.114000'}},
 'status_code': 200}